In [8]:
import sys
import json
sys.path.append('/home/kshitij/legaleval-subtask-a')


from models.CNN_BiLSTM import CNN_BiLSTM
from utils import sent2embeddings, label_encode

# Data

In [9]:
with open("../data/dev.json") as json_file:
    data = json.load(json_file)